### Taking Danske Bank PDF statements and converting into DataFrame

Process
- Merge PDF statements in Adobe and export as xls
- Read xlsx from file
- Wrangle data
- Calculate Flat Fees
- Calculate Exchange rate spread being charge (versus Oanda open price)

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

In [ ]:
df = pd.read_excel('master.xlsx')
cols = [2,5]
df.drop(df.columns[cols], axis =1, inplace = True)

In [ ]:
df.columns

In [ ]:
#rename columns
df.rename(columns = {'NAME AND ADDRESS from the statement': 'meta', 'Unnamed: 4' : 'money'}, inplace =True )

In [ ]:
#add values to cells we dont want to a delete
df.loc[df['meta'].str.contains("Gebyr"), 'Unnamed: 1'] = 0.03
df.loc[df['meta'].str.contains("Der er den"), 'money'] = 0.01
df.loc[df['meta'].str.contains("Der er den"), 'Unnamed: 1'] = 0.01
df.loc[df['meta'].str.contains("Der er den"), 'Unnamed: 3'] = 0.01
df.loc[df['meta'].str.contains("Modtaget beløb"), 'Unnamed: 1'] = 0.02


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns = ['index'], inplace = True)

In [ ]:
df.replace({"Gebyr" : "fee"}, inplace = True)

### Calculating the flat fee total

In [ ]:
flat_fee = df[df['meta'] == "fee"]

In [ ]:
flat_fee['money'] = flat_fee['money'].str.replace(',', '.').astype(float)


In [ ]:
total_fee = flat_fee['money'].sum() / 6.65
f'The amount you are paying in one year in flat fees is {total_fee}'

### Further wrangling to get to spread data

In [ ]:
df.rename(columns = {'Unnamed: 1': 'Ex_Rate', 'Unnamed: 3' : 'currency'}, inplace =True )

In [ ]:
#remove commas from the bank statement and replace with decimals
df['money'] = df['money'].str.replace('.', '')
df['money'] = df['money'].str.replace(',','.').astype(float)

In [ ]:
#removes the rows where the meta column contains the word fee (which removes an obsolete row)
df = df[~df['meta'].str.contains('fee')]

In [ ]:
#translates english months to danish
months = GoogleTranslator(source='auto', target='da').translate("January February March April May June July August September October November December")

In [ ]:
#makes all letters lower case
months = months.lower()

In [ ]:
#convert months string to a list from a string
months = months.split()

In [ ]:
type(months)

In [ ]:
months

In [ ]:
En_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [ ]:
#merge months into a dictionary if needed later 
months_dict = dict(zip(months, En_months))

In [ ]:
#create a new column called months and extract the month from the meta column text and instert into it
df['month'] = df.meta.str.extract('({})'.format('|'.join(months)), expand=False).str.lower().fillna('')
#using extract() followed by the {} and .format enables the months list to be used dynamically, i.e it save us
#typing each month into the line of code, and it separates them using the OR | line.

In [ ]:
#create a new column called year and extract the year from the meta column text and instert into it
year = ['2022', '2023']
df['year'] = df.meta.str.extract('({})'.format('|'.join(year)), expand=False).str.lower().fillna('')

In [ ]:
#bump the Ex_Rate rows down by 1 to get the exchange rate offered by the bank in line with the rest of the row
df.loc[len(df.index), :] = None #this adds a row to the bottom of the data frame and may note be needed.
df['Ex_Rate'] = df.Ex_Rate.shift(1) #this moves everything down one row, adding NAN in the new cell at the top.

#### Pull USD and DKK money in into their own data tables to merge back into df cols later

In [ ]:
cash_in = df['money']

In [ ]:
cash_in.dropna(inplace=True)

In [ ]:
USD = cash_in[::2] #every other number starting with the first number to isolate USD data
shape = USD.shape
datatype = type(USD)
print(f' the shape is {shape} and the data type is {datatype}')

In [ ]:
USD = USD.to_frame() #put it in a dataframe
type(USD)

In [ ]:
USD.reset_index(drop=True, inplace = True) #drop removes the old index from the cols

In [ ]:
#Do the same but with DKK
DKK = cash_in[1::] #skips first row
DKK = DKK[::2] #takes every other number including the first number which is now DKK since USD is skipped in line above
DKK.shape

In [ ]:
DKK.to_frame()
DKK.reset_index(drop=True, inplace = True)

#### Perpare original df to merge with new USD and DKK columns

In [ ]:
df_filter_1 = df[2::]
df_filter_2 = df_filter_1[::3]
df_filter_2.shape #check shape of df matches DKK and USD

In [ ]:
df_filter_2.drop(columns = ['currency', 'money'], inplace=True)

In [ ]:
df_filter_2.reset_index(drop=True, inplace = True)

In [ ]:
#add USD col to the new edited df
df_filter_3 = pd.concat([df_filter_2, USD], axis = 1)

In [ ]:
#update col name
df_filter_3.rename(columns = {'money':'USD'}, inplace = True)

In [ ]:
#add DKK to the df 
df_filter_4 = pd.concat([df_filter_3, DKK], axis = 1)

In [ ]:
df_filter_4.rename(columns = {'money':'DKK'}, inplace = True)

In [ ]:
df = df_filter_4

In [ ]:
#double check the statement matth works out - double check col should show zero
df['double_check'] = DKK - df.USD * (df.Ex_Rate/100) 
df['double_check'].round(decimals = 0)

In [ ]:
#Oanda exchange rates from Trading View for each row beginning at index 0. This is DKK for 1 USD. 
#Free forex data can be pulled in dynamically but since i only had a few did it manually.
U_ex = [6.82519, 6.88532, 6.95565, 6.89235, 6.75840, 6.75145, 6.74435, 6.86866, 7.05846, 6.84380, 6.84235, 6.85434, 7.06845, 7.05008, 7.01252, 7.1453, 7.20540,7.46628, 7.63778, 7.43880, 7.48095] 

In [ ]:
U_ex_df = pd.DataFrame(U_ex, columns = ["Oanda Open"])

In [ ]:
U_ex_df.shape

In [ ]:
df = pd.concat([df, U_ex_df], axis = 1)

In [ ]:
#add how much dkk you would be paid based on the interbank rate to a new col
df["interbank"] = df.USD * df['Oanda Open']

In [ ]:
#calculate the spread being charged by statement(ie by row)
df['Spread_DKK'] = df.interbank - df.DKK
df['fee'] = (df.Spread_DKK / df.interbank) * 100
df

In [ ]:
#sum the spread fee column
spread_fee = df['Spread_DKK'].sum()

In [ ]:
total_sal = df.DKK.sum()

#### Add data to Summary dataframe

In [ ]:
summary = pd.DataFrame({"flat_fees": total_fee, "spread_fees": spread_fee, "total_fees" : total_fee+spread_fee}, index=[0])
summary["percent_of_sal"] = summary.total_fees / total_sal * 100
summary

In [ ]:
# Unhash to get list of google translator supported languages
# GoogleTranslator().get_supported_languages()
